# import magic
* **Raúl Cumplido**
* **@raulcumplido**
* **PyConES 2015 - Valencia**

![caption](img/te_chie_la.png)

# Versions

* Almost everything **Python 3.4**
* Some bits **Python 3.5**
* Extra **Python 3.6**

# Covered
* Diving deep on how "straightforward" import works

# Not covered
* Packages
* NamespacePaths
* Importlock
* Lots of other stuff going on behind the scences (corner cases)

In [1]:
import math
math.sin(math.pi/2)

1.0

In [2]:
import collections
collections.OrderedDict()

OrderedDict()

# Easy Peasy

![caption](img/import_1.png)

In [ ]:
# collections.py, math.py and my_method.py
def my_method():
    print(__name__)

>>> import my_module
>>> my_module.my_method()
my_module
>>> import math
>>> import collections
>>> math.my_method()
math
>>> collections.my_method()
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
AttributeError: module 'collections' has no attribute 'my_method'
>>> collections.OrderedDict()
OrderedDict()
>>> 

.
├── collections.py
├── math.py
├── my_module.py
└── __pycache__
    ├── math.cpython-35.pyc
    └── my_module.cpython-35.pyc

# WAT!!!

![caption](img/wat.gif)

# Import machinery

* Importlib is the implementation of import

In [25]:
import importlib
print(importlib.machinery.SOURCE_SUFFIXES)
print(importlib.machinery.BYTECODE_SUFFIXES)
print(importlib._bootstrap_external.MAGIC_NUMBER)
importlib.import_module("collections")

['.py']
['.pyc']
b'\x16\r\r\n'


<module 'collections' from '/home/raulcd/.virtualenvs/import_magic/lib/python3.5/collections/__init__.py'>

# sys.modules: Caching modules

![caption](img/import_3.png)

# Cache All The Things!

![caption](img/cache.jpg)

# Find spec and load module

![caption](img/import_4.png)

# sys.meta_path: Importers

![caption](img/import_5.png)

In [26]:
import sys
sys.meta_path

[_frozen_importlib.BuiltinImporter,
 _frozen_importlib.FrozenImporter,
 _frozen_importlib_external.PathFinder]

In [ ]:
@host - ~$ python -vv
import _frozen_importlib # frozen
import imp # builtin
import sys # builtin
import '_warnings' # <class '_frozen_importlib.BuiltinImporter'>
import '_thread' # <class '_frozen_importlib.BuiltinImporter'>
import '_weakref' # <class '_frozen_importlib.BuiltinImporter'>
import '_frozen_importlib_external' # <class '_frozen_importlib.FrozenImporter'>
import '_io' # <class '_frozen_importlib.BuiltinImporter'>
import 'marshal' # <class '_frozen_importlib.BuiltinImporter'>
import 'posix' # <class '_frozen_importlib.BuiltinImporter'>
import _thread # previously loaded ('_thread')
import '_thread' # <class '_frozen_importlib.BuiltinImporter'>
import _weakref # previously loaded ('_weakref')
import '_weakref' # <class '_frozen_importlib.BuiltinImporter'>
# installing zipimport hook
import 'zipimport' # <class '_frozen_importlib.BuiltinImporter'>


# sys.meta_path: BuiltinImporter

![caption](img/import_6.png)

# sys.meta_path: FrozenImporter

![caption](img/import_7.png)

# PathFinder: The finder searchs a finder

![caption](img/import_8.png)

# The finder searchs a finder

![caption](img/infinite.gif)

# PathFinder: The fun begins

![caption](img/import_9.png)

In [ ]:
>>> import sys
>>> sys.path_importer_cache
{'/usr/lib/python35.zip': None, '/usr/lib/python3.5/plat-linux': FileFinder('/usr/lib/python3.5/plat-linux'), 
 '/usr/lib/python3.5/site-packages': FileFinder('/usr/lib/python3.5/site-packages'), 
 '/usr/lib/python3.5': FileFinder('/usr/lib/python3.5'), '/usr/lib/python3.5/': FileFinder('/usr/lib/python3.5/'), 
 '/usr/lib/python3.5/lib-dynload': FileFinder('/usr/lib/python3.5/lib-dynload'), 
 '/usr/lib/python3.5/encodings': FileFinder('/usr/lib/python3.5/encodings'), 
 '/usr/lib/python3.5/collections': FileFinder('/usr/lib/python3.5/collections')}

In [34]:
import sys
sys.path_hooks

[zipimport.zipimporter,
 <function _frozen_importlib_external.FileFinder.path_hook.<locals>.path_hook_for_FileFinder>]

In [43]:
# /test.py exists otherwise None is returned
sys.path_hooks[1]('/').find_spec('test')

ModuleSpec(name='test', loader=<_frozen_importlib_external.SourceFileLoader object at 0x7f43f4035e80>, origin='/test.py')

In [ ]:
>>> spec = sys.meta_path[2].find_spec('collections')
# trying /home/raulcd/collections.cpython-35m-x86_64-linux-gnu.so
# trying /home/raulcd/collections.abi3.so
# trying /home/raulcd/collections.so
# trying /home/raulcd/collections.py
# trying /home/raulcd/collections.pyc
>>> dir(spec)
[... 'cached', 'has_location', 'loader', 'loader_state', 'name', 'origin', 'parent', 'submodule_search_locations']
>>> spec.name
'collections'
>>> spec.parent
'collections'
>>> spec.cached
'/usr/lib/python3.5/collections/__pycache__/__init__.cpython-35.pyc'
>>> spec.loader
<_frozen_importlib_external.SourceFileLoader object at 0x7f8240091be0>
>>> 

# Recap: find_spec

![caption](img/import_10.png)

# Find spec and load module

![caption](img/import_4.png)

# Load module

![caption](img/import_11.png)

# Create module object

![caption](img/import_12.png)

In [ ]:
def _new_module(name):
    return type(sys)(name)

# Load module: Context Manager and exec_module

![caption](img/import_11.png)

In [ ]:
class _installed_safely:
    ...

    def __enter__(self):
        ...
        sys.modules[self._spec.name] = self._module

In [ ]:
#foo.py
import bar

def name():
    print(__name__)

bar.name()

In [ ]:
# bar.py
import foo

foo.name()

def name():
    print(__name__)

In [ ]:
$ python
Python 3.5.0 (default, Sep 20 2015, 11:28:25) 
[GCC 5.2.0] on linux
Type "help", "copyright", "credits" or "license" for more information.
>>> import foo
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
  File "/home/raulcd/import_magic/_examples/import_reference_1/foo.py", line 1, in <module>
    import bar
  File "/home/raulcd/import_magic/_examples/import_reference_1/bar.py", line 3, in <module>
    foo.name()
AttributeError: module 'foo' has no attribute 'name'

In [ ]:
# foo.py
import bar

def name():
    print(__name__)

bar.name()

In [ ]:
# bar.py
import foo

foo.__name__

def name():
    print(__name__)
    foo.name()

In [ ]:
$ python
Python 3.5.0 (default, Sep 20 2015, 11:28:25) 
[GCC 5.2.0] on linux
Type "help", "copyright", "credits" or "license" for more information.
>>> import foo
bar
foo
>>> 

# Load module: Context Manager and exec_module

![caption](img/import_11.png)

# get_code

![caption](img/import_13.png)

In [44]:
from importlib import _bootstrap_external
_bootstrap_external.cache_from_source('/tomato.py')

'__pycache__/tomato.cpython-35.pyc'

In [46]:
_bootstrap_external.cache_from_source('/tomato.py', optimization=1)

'__pycache__/tomato.cpython-35.opt-1.pyc'

In [47]:
_bootstrap_external.cache_from_source('/tomato.py', optimization=2)

'__pycache__/tomato.cpython-35.opt-2.pyc'

In [ ]:
python -O
python -OO
def test():
    """
    This is my test function --> Removed on optimization 2
    """
    assert False == True --> Removed on optimization 1 and 2

# get_code

![caption](img/import_13.png)

In [63]:
compile("def my_func():\n    print(__name__)", "string", "exec")

<code object <module> at 0x7f43e77d30c0, file "string", line 1>

In [71]:
exec(code, globals())
my_func()

__main__


In [72]:
import dis
dis.dis(code)

  1           0 LOAD_CONST               0 (<code object my_func at 0x7f43f400c150, file "string", line 1>)
              3 LOAD_CONST               1 ('my_func')
              6 MAKE_FUNCTION            0
              9 STORE_NAME               0 (my_func)
             12 LOAD_CONST               2 (None)
             15 RETURN_VALUE


# Load code

![caption](img/import_14.png)

# Starting point

![caption](img/import_1.png)

# End

![caption](img/import_15.png)

# To remember

* Specs
 * Definition of how to load a module and metadata of the module
* Finders
 * Strategies to find the module and specify a Loader.
 * Creation of Spec
* Loaders
 * Load a module
 * Insert it in sys.modules

# Extend import

* sys.meta_path
 * New Finders (i.e. HTTPFinder, FTPFinder)
* sys.path_hooks
 * New PathFinders (i.e. XMLFinder, GoFinder)
* sys.modules
 * Cache of modules
* sys.path
 * Where the finders are going to take a look on